# Joining files together

The purpose of this file is to convey methodology and logic that can be used to join our files together.  To this end, we will cover each our of Raw 2.0 files using a slice of the full job records.  These methodologies can be scaled up in a SQL data warehouse or by chunking the files depending on the subset you are working with.

In [1]:
import pandas as pd
import sqlite3
from xml.etree import ElementTree as ET

# Load Job Records

### Job records is the core table.  Everything will be joined into Job Records.

##### I will only be using a small subset of the columns for demonstration purposes

In [2]:
# Loading raw sample slice
with tarfile.open('../../../Feeds/raw-sample.tar.gz', "r:*") as tar:
    # Get path to the job_records file within the tarfile
    csv_path = tar.getnames()[1]
    # Load job records file into pandas dataframe
    job_records = pd.read_csv(tar.extractfile(csv_path),
                    parse_dates = ['created','delete_date','last_checked','last_updated'],
                    low_memory = False,
                    usecols = ['hash','title','company_id','created','delete_date'])

# Join fs_company_reference

##### I am only joining primary information, and only based on created date as that is a good starting point.  Please reach out to Linkup if you would like to discuss options for joining to use information on multiple dates to account for jobs that span changes/M&A activity.

In [14]:
# Read PIT Ticker File
PIT_Ticker = pd.read_csv('../../../Feeds/raw-sample/raw_pit_ticker_full.csv.gz',
                        parse_dates = ['start_date','end_date'])
                      
# Filter for only primary exchange and columns needed for join
PIT_Ticker = PIT_Ticker[PIT_Ticker.primary_flag == True]

# Formatting Timestamps for Merge and dealing with missing values
PIT_Ticker['start_date'] = PIT_Ticker['start_date'].fillna(str(Job_Records.created.min().date()))
PIT_Ticker['start_date'] = pd.to_datetime(PIT_Ticker['start_date'])
PIT_Ticker['end_date'] = PIT_Ticker['end_date'].fillna(str(Job_Records.created.max().date()))
PIT_Ticker['end_date'] = pd.to_datetime(PIT_Ticker['end_date'])

# Make the db in memory
conn = sqlite3.connect(':memory:')

# Write the tables
Job_Records.to_sql('Job_Records', conn, index=False)
PIT_Ticker.to_sql('PIT_Ticker', conn, index=False)

# Query and create new joined table
qry = '''
    SELECT Job_Records.*,
    PIT_Ticker.stock_ticker,
    PIT_Ticker.stock_exchange_country,
    PIT_Ticker.stock_exchange_name,
    PIT_Ticker.primary_flag
    
    FROM Job_Records
    
    LEFT JOIN PIT_Ticker
    ON (Job_Records.created between PIT_Ticker.start_date and PIT_Ticker.end_date and
       Job_Records.company_id = PIT_Ticker.company_id);
    '''
job_records_joined = pd.read_sql_query(qry, conn)

PIT_Ticker = None

# Join PIT Company Reference

##### For the PIT Company Reference File, most use-cases only require joining the most current company information to all records historically.  If you would like to use the PIT nature of the file, please reach out to Linkup and we would be happy to provide assistance with that.

In [15]:
PIT_Company_Reference = pd.read_csv('../../../Feeds/raw_pit_company_reference_full_2019-11-01.csv.gz')

# Filter Company Reference file to include only the companies that we pulled
#PIT_Company_Reference = PIT_Company_Reference[PIT_Company_Reference.company_id.isin(job_records.company_id.unique())]

# Filter Company Reference for the latest Information
#PIT_Company_Reference = PIT_Company_Reference[PIT_Company_Reference.end_date.isnull()]

# Make the db in memory
conn = sqlite3.connect(':memory:')

# Write the tables
job_records.to_sql('job_records', conn, index=False)
PIT_Company_Reference.to_sql('PIT_Company_Reference', conn, index=False)

qry = '''
    SELECT 
        Job_Records.*, 
        PIT_Company_Reference.company_url,
        PIT_Company_Reference.lei,
        PIT_Company_Reference.open_perm_id,
        
    FROM job_records
    LEFT JOIN PIT_Company_Reference
    ON job_records.company_id = PIT_Company_Reference.company_id and
        job_records.created BETWEEN 
            PIT_Company_Reference.start_date and 
            PIT_Company_Reference.end_date
        ;
'''

job_records_joined = pd.read_sql_query(qry, conn)

PIT_Company_Reference = None

# Join Scrape Log

##### Here I will do a basic join to show the date of the next scrape change.  The primary purpose of this is so that if you see outliers, you can see when the next code change was.  If it was shortly after the outlier, the outlier is likely a scrape break vs a true signal.

In [45]:
# Load Scrape Log
Scrape_Log = pd.read_csv('../raw-sample/raw_company_scrape_log_full.csv.gz')

# Filter Scrape Log to only include code changes for the companies that we pulled
Scrape_Log = Scrape_Log[Scrape_Log.scrape_changed == True]
Scrape_Log = Scrape_Log[Scrape_Log.company_id.isin(Job_Records.company_id.unique())]

In [47]:
# Shift Date for SQL query convenience
Scrape_Log['date_shifted'] = Scrape_Log.groupby(['company_id'])['date'].shift(1)
Scrape_Log['date_shifted'] = Scrape_Log['date_shifted'].fillna('2007-01-01')

Scrape_Log['date_shifted'] = pd.to_datetime(Scrape_Log['date_shifted'])
Scrape_Log['date'] = pd.to_datetime(Scrape_Log['date'])
# Make the db in memory
conn = sqlite3.connect(':memory:')

# Write the tables
Job_Records.to_sql('Job_Records', conn, index=False)
Scrape_Log.to_sql('Scrape_Log', conn, index=False)

# Query and create new joined table
qry = '''
    SELECT Job_Records.*,
    Scrape_Log.date as Next_Scrape_Change
    
    FROM Job_Records
    
    LEFT JOIN Scrape_Log
    ON (Job_Records.created between Scrape_Log.date_shifted and Scrape_Log.date and
       Job_Records.company_id = Scrape_Log.company_id);
    '''
job_records_joined = pd.read_sql_query(qry, conn)

Scrape_Log = None

# Join Description

In [18]:
# Load Descriptions file
Job_Descriptions = pd.DataFrame(
        list(map(lambda x: (x[0].text,x[1].text),
                 ET.parse('../raw-sample/raw-sample-descriptions.xml').getroot())),       
        columns = ['job_hash','description'])

# Make the db in memory
conn = sqlite3.connect(':memory:')

# Write the tables
Job_Records.to_sql('Job_Records', conn, index=False)
Job_Descriptions.to_sql('Job_Descriptions', conn, index=False)

qry = '''
    SELECT 
        Job_Records.*, 
        Job_Descriptions.description
    
    
    FROM Job_Records
    LEFT JOIN Job_Descriptions
    ON (Job_Records.hash = Job_Descriptions.job_hash);
'''
job_records_joined = pd.read_sql_query(qry, conn)

Job_Descriptions = None

# View Results of Big Dataframe

In [50]:
job_records_joined[~job_records_joined.description.isnull()]

,hash,title,company_id,city,state,created,delete_date,start_date,end_date,stock_ticker,stock_exchange_country,stock_exchange_name,primary_flag,company_url,lei,open_perm_id,naics_code,description,Next_Scrape_Change
2,0234422f9c5eff7f6d1d008c3c31dae6,"UI Artist, Double Helix Games, Amazon Game Stu...",469,Irvine,CA,2015-06-20 02:39:07+00:00,2015-09-18 07:51:10+00:00,1997-05-15 00:00:00,2019-03-31 00:00:00,AMZN,US,NAS,1.0,http://www.amazon.com,ZXTILKJKG63JELOEG630,4295905494,454110.0,Amazon is all in on games.\n\nWe believe the e...,2017-01-30 00:00:00
7,0c392d37eb6dfb15b1ffc2a48b8d95e7,"Warehouse Team Member (Seasonal, Part Time, Fl...",469,englewood,CO,2019-01-15 16:06:00+00:00,None,1997-05-15 00:00:00,2019-03-31 00:00:00,AMZN,US,NAS,1.0,http://www.amazon.com,ZXTILKJKG63JELOEG630,4295905494,454110.0,"Shifts:\n\nOver-night, Early Morning, Day, Eve...",2019-07-26 00:00:00
8,0ee6d7263a67121e8405af8ad8fdef2a,Responsable des opérations de contrôle d'inven...,469,None,None,2018-04-17 15:04:00+00:00,2018-04-18 15:05:00+00:00,1997-05-15 00:00:00,2019-03-31 00:00:00,AMZN,US,NAS,1.0,http://www.amazon.com,ZXTILKJKG63JELOEG630,4295905494,454110.0,ous aimez l'action ?\n\nVous aimeriez travaill...,2019-07-26 00:00:00
9,0aaf97d36a5279ac18a43d1e34344232,"Relationship Manager-Newton/Waltham, MA Area",381,Newton,MA,2015-10-15 20:58:31+00:00,2015-10-23 06:54:04+00:00,1998-10-01 00:00:00,2019-03-31 00:00:00,BAC,US,NYS,1.0,http://www.bankofamerica.com,9DJT3UXIJIZJI4WXO774,8589934339,522110.0,"Located in a banking center, Relationship Mana...",2015-10-23 00:00:00
13,05e060a4db459d502ed71fc78ea75f62,Database Administrator II - AMZ1781,469,Seattle,WA,2017-07-02 09:46:00+00:00,2017-07-22 19:00:00+00:00,1997-05-15 00:00:00,2019-03-31 00:00:00,AMZN,US,NAS,1.0,http://www.amazon.com,ZXTILKJKG63JELOEG630,4295905494,454110.0,MULTIPLE POSITIONS AVAILABLE Company: Amazon C...,2017-07-14 00:00:00
14,042115563f45903598ecc13e26e5f8bd,Relationship Manager (Retail Banking) FOOTHILL...,381,Arcadia,CA,2017-01-25 01:57:15+00:00,2017-04-27 12:27:18+00:00,1998-10-01 00:00:00,2019-03-31 00:00:00,BAC,US,NYS,1.0,http://www.bankofamerica.com,9DJT3UXIJIZJI4WXO774,8589934339,522110.0,Job Description:\n\nLocated in a banking cente...,2017-01-30 00:00:00
17,0f8de29e71cc0c58ca58d82b0e98198f,Sr. Business Intelligence Engineer,469,seattle,WA,2019-01-17 16:07:00+00:00,2019-01-19 16:07:00+00:00,1997-05-15 00:00:00,2019-03-31 00:00:00,AMZN,US,NAS,1.0,http://www.amazon.com,ZXTILKJKG63JELOEG630,4295905494,454110.0,Want to transform the way people enjoy video? ...,2019-07-26 00:00:00
18,08f612e8762a31ce19e790674659423d,Teller - So MD/Eastern Shores Market - La Plat...,381,La Plata,MD,2015-05-08 08:27:09+00:00,2015-05-29 03:50:06+00:00,1998-10-01 00:00:00,2019-03-31 00:00:00,BAC,US,NYS,1.0,http://www.bankofamerica.com,9DJT3UXIJIZJI4WXO774,8589934339,522110.0,Responsibilities : Professional tellers are re...,2015-05-19 00:00:00
19,0a1ce2aef8b7e87520a03c8b82528de4,".Net Developer - C#, WCF & SQL Server",381,charlotte,NC,2018-01-22 21:11:00+00:00,2018-02-22 09:42:00+00:00,1998-10-01 00:00:00,2019-03-31 00:00:00,BAC,US,NYS,1.0,http://www.bankofamerica.com,9DJT3UXIJIZJI4WXO774,8589934339,522110.0,"Job Description:\n\nDevelops, enhances, debugs...",None
20,09b2f4d92c41899602326ff2835ccc44,Relationship Manager - Scottsdale Promenade - ...,381,Scottsdale,AZ,2017-06-21 05:08:00+00:00,2017-06-30 06:42:00+00:00,1998-10-01 00:00:00,2019-03-31 00:00:00,BAC,US,NYS,1.0,http://www.bankofamerica.com,9DJT3UXIJIZJI4WXO774,8589934339,522110.0,Job Description:\n\nRelationship managers are ...,None
